<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#Futures" data-toc-modified-id="Futures-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Futures</a></span><ul class="toc-item"><li><span><a href="#Callbacks" data-toc-modified-id="Callbacks-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Callbacks</a></span></li></ul></li></ul></div>

In [1]:
#include <mutex>
#include <string>
#include <unordered_set>
#include <functional>
#include <condition_variable>
#include <deque>
#include <thread>
#include <iostream>

using namespace std;

In [2]:
namespace bcc {

class sequential_process {
    using task = function<void()>;

    mutex _mutex;
    condition_variable _condition;
    deque<task> _queue;
    bool _done = false;

    void run_loop();

    thread _thread{[this] { run_loop(); }};

public:
    ~sequential_process();
    void async(task f);
};
    
sequential_process::~sequential_process() {
    {
        lock_guard<mutex> lock(_mutex);
        _done = true;
    }
    _condition.notify_one();
    _thread.join();
}

void sequential_process::async(task f) {
    {
        lock_guard<mutex> lock(_mutex);
        _queue.push_back(move(f));
    }
    _condition.notify_one();
}

void sequential_process::run_loop() {
    while (true) {
        task work;
        {
            unique_lock<mutex> lock(_mutex);

            while (_queue.empty() && !_done) {
                _condition.wait(lock);
            }

            if (_queue.empty()) return;

            work = move(_queue.front());
            _queue.pop_front();
        }
        work();
    }
}
    
} // namespace bcc

using namespace bcc;

# Futures

- Homework from last class, rewrite interned_string as a sequential process

In [3]:
class interned_string {
    struct shared_pool {
        mutex _mutex;
        unordered_set<string> _pool;

        const string* insert(const string& a) {
            lock_guard<mutex> lock(_mutex);
            return &*_pool.insert(a).first;
        }
    };

    static auto pool() -> shared_pool& {
        static shared_pool result;
        return result;
    }

    const std::string* _string;
public:
    interned_string(const string& a) : _string(pool().insert(a)) {}
    const string& str() const { return *_string; }
};

In [4]:
.undo 1

```cpp
struct shared_pool {
    unordered_set<string> _pool;
    sequential_process _process;

    const string* insert(const string& a) {
        _process.async([&, _a = a]{
            _pool.insert(a).first;
        });

        return ???;
    }
};
```

## Callbacks

In [5]:
struct shared_pool {
    sequential_process _process;
    unordered_set<string> _pool;

    template <class F> // F models void(const string*)
    void insert(string a, F&& f) {
        _process.async([this, _a = move(a), _f = forward<F>(f)]{
            _f(&*_pool.insert(_a).first);
        });
    }
};

```cpp
class interned_string {
    // struct shared_pool

    static auto pool() -> shared_pool& {
        static shared_pool result;
        return result;
    }

    const std::string* _string;
public:
    interned_string(const string& a) : _string(pool().insert(a, ???)) {}
    const string& str() const { return *_string; }
};
```

In [6]:
class interned_string {
    // struct shared_pool

    static auto pool() -> shared_pool& {
        static shared_pool result;
        return result;
    }

    const std::string* _string;
    
    interned_string(const string* s) : _string(s) {}
public:
    template <class F> // F models void(interned_string)
    static void make(string a, F&& f) {
        pool().insert(move(a), [_f = forward<F>(f)](const string* s){
            _f(interned_string(s));
        });
    }
    const string& str() const { return *_string; }
};

In [7]:
interned_string::make("Hello World"s, [](const interned_string& s){
    cout << s.str() << endl;
});

this_thread::sleep_for(1s);

Hello World


- Pros of callbacks:
    - Fast, no synchronization required
    - Easy to understand
- Cons
    - Requires code be transformed into functional form
    - You must know were a value is going before invocation
    - Challenging to make exception safe

[Part 2 - notebook](./11-futures-pt2.ipynb)

[Part 2](./11-futures-pt2.slides.html)